In [3]:
## Primeiro vamos importar as bibliotecas iniciais para realizar a raspagem de dados, vamos começar com a BeautifulSoup para ajudar na estruturação do HTML que iremos utilizar. 
## A urllib.request é para fazer as requisições de URL's que serão utilizadas para alimentar o HTML ao BeautifulSoup.

In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [4]:
## Vamos definir nossa primeira variável, sendo o URL que iremos fazer nossas requisições iniciais.
link = 'https://www.camaraindaial.sc.gov.br/pg/proposicoes'

In [11]:
## Agora vamos passar nosso link que foi definido acima, para o parser.
html = urlopen(link)
soup = BeautifulSoup(html, 'html.parser')

In [13]:
## Caso desejamos, conseguimos analisar o código fonte da página em questão, para entender um pouco mais da estrutura da mesma.
print(soup.prettify)

<bound method Tag.prettify of 
<!DOCTYPE html>

<html class="h-100">
<head>
<!--GENERAL-->
<meta charset="utf-8"/>
<meta content="width=device-width,minimum-scale=1,initial-scale=1" name="viewport"/>
<!--PAGE TYPE-->
<meta content="website" property="og:type"/>
<meta content="summary_large_image" property="twitter:card"/>
<!--TITLE-->
<title>Proposições   |  Câmara Municipal de Indaial</title>
<meta content="Proposições" property="twitter:title"/>
<meta content="Proposições" property="og:title"/>
<!--SUBTITLE-->
<meta content="" property="twitter:description"/>
<meta content="" property="og:description"/>
<!--DESCRIPTION-->
<meta content="Conteúdo estático com links para o Legislador." property="description"/>
<!--IMGS-->
<meta content="https://www.camaraindaial.sc.gov.br/view/dist/imgs/metatag.jpg" property="twitter:image"/>
<meta content="https://www.camaraindaial.sc.gov.br/view/dist/imgs/metatag.jpg" property="twitter:image:alt"/>
<meta content="https://www.camaraindaial.sc.gov.br/v

[<p><strong><a href="http://www.legislador.com.br/LegisladorWEB.ASP?WCI=ProposicaoConsulta&amp;ID=3&amp;tpProposicao=1&amp;" target="_blank"><span style="color: rgb(192, 80, 77);"><span style="font-size: 18px;">Indicações</span> </span><span style="font-size: 14px; color: rgb(192, 80, 77);">(pesquise aqui)</span></a></strong><br/></p>,
 <p>Proposições escritas, por meio das quais o vereador sugere medidas de interesse público aos poderes competentes. </p>,
 <p><br/></p>,
 <p><strong><a href="http://www.legislador.com.br/LegisladorWEB.ASP?WCI=ProposicaoConsulta&amp;ID=3&amp;tpProposicao=3&amp;" target="_blank"><span style="font-size: 18px;"></span></a><a href="http://www.legislador.com.br/LegisladorWEB.ASP?WCI=ProposicaoConsulta&amp;ID=3&amp;tpProposicao=3&amp;" target="_blank"><span style="color: rgb(192, 80, 77);"><span style="font-size: 18px;">Moções</span> <span style="font-size: 14px;">(pesquise aqui)</span></span></a><span style="font-size: 14px;"></span><span style="font-size: 14